# Mechanisms of Action Project Submission

Team Members: Abhi Gandhi, Jeff Won, Shashank Saurav, Shengebo Zhang, Faraz Khoshbakhtian

This document outlines the steps taken by our group to approach the Mechanisms of Action competition. This document presents are original approach whereas another document presents the challanger approach. 

## Importing the needed libraries

In [22]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


### This part is to run pyspark locally
import findspark  # Get rid of this in DataBricks
findspark.init('/opt/spark-3.0.1')  # Get rid of this in DataBricks #faraz: you can remove the parameter. it only worked like this for me
########################################


from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F 
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression, NaiveBayes
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.functions import vector_to_array


from tqdm import tqdm

## Seeting up spark, contexts and configurations

In [5]:
config = SparkConf().setAll([('spark.executor.memory', '30g'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])
config.setAppName("proj")
config.set("spark.dynamicAllocation.minExecutors", "2");
config.set("spark.dynamicAllocation.maxExecutors", "2");
config.set("spark.dynamicAllocation.initialExecutors", "2"); # the number must be between the min and max
sc = SparkContext(conf=config)  # start a new sc with the current config
spark = SparkSession(sc)
sqlc=SQLContext(sc)
print(sc.getConf().getAll())  # print all the configuration

[('spark.dynamicAllocation.initialExecutors', '2'), ('spark.app.id', 'local-1607296188739'), ('spark.executor.memory', '30g'), ('spark.executor.id', 'driver'), ('spark.dynamicAllocation.minExecutors', '2'), ('spark.driver.host', '192.168.2.11'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.dynamicAllocation.maxExecutors', '2'), ('spark.rdd.compress', 'True'), ('spark.driver.memory', '8g'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.app.name', 'proj'), ('spark.driver.port', '60519')]


## Reading and Preprocessing the Train Data

### Reading and Joining dataframes

First, we need to read the features and labels for the training data, get rid of controll cases and join all the training data into a singular Spark dataframe.

In [11]:
# This is for being able to store the data in github and concatenation on local computer
!cat train_features_*.csv > train_feats.csv

In [7]:
# Train Features Dataset
df_train = spark.read.csv('train_feats.csv', header='true', inferSchema= 'true')   # path in HDFS file system

In [8]:
# Labels Dataset
df_label = spark.read.csv('train_targets_scored.csv', header='true', inferSchema= 'true')

In [9]:
# join all data together
df = df_train.join(df_label, on=['sig_id'], how='left_outer')  

In [12]:
# Dropping all the control cases since the test data also comes with control/case flag
df = df.filter(df.cp_type == 'trt_cp')
df = df.drop('cp_type')

### One-Hot Encoding for Categorical features

Second, we need to **One-Hot Encode** our categorical features. 

In [13]:
# String indexer for cp_dose
indexer = StringIndexer(inputCol="cp_dose", outputCol="cp_dose_cat")
# index cp_dose in data
df1 = indexer.fit(df).transform(df)

# String indexer for cp_time
indexer = StringIndexer(inputCol="cp_time", outputCol="cp_time_cat")
# index cp_time in data
df1 = indexer.fit(df1).transform(df1)
df1 = df1.drop('cp_dose')
df1 = df1.drop('cp_time')

# One-hot enocder 
encoder = OneHotEncoder(inputCols=["cp_time_cat", "cp_dose_cat"],
                        outputCols=["cp_time_onehot", "cp_dose_onehot"])

model = encoder.fit(df1)
df1 = model.transform(df1)

# Keeping n-1 dummy variables for each feature. (dummy variables have degree of fredom n-1)
df1 = df1.withColumn("cp_time_cols", vector_to_array("cp_time_onehot")).select(df1.columns + [col("cp_time_cols")[i] for i in range(2)])
df1 = df1.withColumn("cp_dose_cols", vector_to_array("cp_dose_onehot")).select(df1.columns + [col("cp_dose_cols")[i] for i in range(1)])

# drop leftover cols
df1 = df1.drop('cp_dose_cat',
 'cp_time_cat',
 'cp_time_onehot',
 'cp_dose_onehot',
)


### Feature Engineering  

Third, we create some features such as min, max row-wise for gene and cell information

In [14]:
gene_feature_names = [name for name in df1.columns if 'g-' in name]
cell_feature_names =  [name for name in df1.columns if 'c-' in name]

df2 = df1.withColumn("gene_max", F.greatest(*gene_feature_names))
df2 = df2.withColumn("gene_min", F.least(*gene_feature_names))
df2 = df2.withColumn("cell_max", F.greatest(*cell_feature_names))
df2 = df2.withColumn("cell_min", F.least(*cell_feature_names))



### General Purpose Feature Selection Based on Correlation

In our initial step for feature selection we use correlation meassure to reduce the number of features.

In [17]:
# Keeping track of the feature names
feature_columns = gene_feature_names + cell_feature_names 

# Creating the feature vector
vectorAssembler = VectorAssembler(inputCols = feature_columns, outputCol = 'feats' )
feature_vector = vectorAssembler.transform(df2).select("feats")

# Computing the correlations with pySpark
corr_matrix = Correlation.corr(feature_vector, "feats").head()[0]

# Convert the correlation desne matrix and apply mask and to get the indicies where high correlations are observed
# In here, I convert the correlation matrix to numpy, and then use numpy's mask to obtain the lower traingle of the
# matrix. I used numpy becasue pyspark does not have mask.

# Detection highly correlated features
corr_Array = corr_matrix.toArray()
masked_corr = np.ma.masked_where(np.triu(np.ones_like(corr_Array, dtype=bool)), corr_Array, copy=True)  
idx_high_corr_feats = set(np.argwhere(abs(masked_corr) > 0.90)[:,0])  # Set threshold to 90%
# Identify the column to drop 
features_to_drop = np.array(feature_columns)[list(idx_high_corr_feats)].tolist()


# Drop the correalted features
df3 = df2.drop(*features_to_drop)

## Machine Learning Pipeline 

The task for Mechanisms of Action is formally considred a Multi-label classification problem. We are employing the Binary-Relevance approach for the problem at hand. For each of the possible binary labels we first use a **Random Forrest CLassifier** to achive another level of feature selection based on the feature importance meassure. Subsequently, we train one **logistic regression** model and a **naive bayes** model for each label.

We have use upsampling the minority class and downsampling the majority class to overcome issues of imbalanced data. 

In [18]:
# Getting the final feature set
final_feature_names = list(set(df3.columns) - set(df_label.columns))

# Create Feature vector
vectorAssembler = VectorAssembler(inputCols = final_feature_names, outputCol = 'feats' )

# drop all unnecessary features
df4 = vectorAssembler.transform(df3).drop(*final_feature_names)

In [19]:
# This function is responsible for running our ml pipeline

#df is the train dataset, label_name is the label we want to do the training on
def train_individual_label(df, label_name, seed = 42):
    
    # we need at least 2 positive instances for this particular label_name
    if df.filter(df[label_name] == 1).count() >= 2:
    
        # dataframe consisting only of the train features and the label name
        temp_df = df.select('feats', label_name)

        # stratify split of the dataframe for train-test split
        fractions = {1: 0.8, 0: 0.8}
        train_df = temp_df.stat.sampleBy(label_name, fractions, seed, )
        test_df =  temp_df.subtract(train_df)


        # Over/down sampling of the training dataframe, becasue of the imbalanced class label
        activation_samples = train_df.filter(train_df[label_name] == 1)
        non_activation_samples = train_df.filter(train_df[label_name] == 0)
        ratio = activation_samples.count() / non_activation_samples.count()

        # somewhat arbitrary ration meassure
        default_down_sample_ratio = 0.5  
        upsample_ratio = default_down_sample_ratio / ratio

        activation_samples_up = activation_samples.sample(True, upsample_ratio, seed)   # Upsample the activation samples
        non_activation_samples_down = non_activation_samples.sample(True, default_down_sample_ratio, seed)  # Meanwhile, downsampling the non-activated samples 

        # randomise the order of samples
        final_train_df = activation_samples_up.union(non_activation_samples_down).orderBy(F.rand())


        # another layer of feature selection using random forrest before training the ml models
        clf = RandomForestClassifier(numTrees=20, maxDepth=5, labelCol = label_name, featuresCol='feats',  seed=42)
        model = clf.fit(final_train_df)
        feature_importance = model.featureImportances.toArray()
        
        # indeces of only the top 10% of features
        important_feature_idx = feature_importance.argsort()[-int(0.1 * len(feature_importance)):]  
                                                                                                   


        # Now, after getting the index, filter the feature vector based on the above feature importance index
        slicer = VectorSlicer(inputCol="feats", outputCol="feats_sub", indices=important_feature_idx)
        final_train_df_sub_feats =  slicer.transform(final_train_df).drop('feats')
        final_test_df_sub_feats = slicer.transform(test_df).drop('feats')



        # Use CV to train a logistic regression model
        lr = LogisticRegression(maxIter=10, featuresCol='feats_sub',  labelCol = label_name)
        lr_paramGrid = ParamGridBuilder() \
                        .addGrid(lr.regParam, [ 0.1, 0.01]) \
                        .addGrid(lr.elasticNetParam, [1,  0])\
                        .build()

        
        lr_evaluator = MulticlassClassificationEvaluator(labelCol=label_name, metricName='logLoss')
        lr_crossval = CrossValidator(estimator=lr,
                              estimatorParamMaps=lr_paramGrid,
                              evaluator=lr_evaluator,
                              numFolds=3)  

        lr_cvModel = lr_crossval.fit(final_train_df_sub_feats)
        lr_prediction_df = lr_cvModel.transform(final_test_df_sub_feats)

        lr_cvModel.save(f"./logistics/{label_name}.model")
        lr_prediction_df.write.save(f"./logistics/{label_name}_prediction_df.parquet", format="parquet")
        final_train_df_sub_feats.write.save(f"./logistics/{label_name}_train_df.parquet", format="parquet")

        lr_log_loss = lr_evaluator.evaluate(test_df)

        with open(f"./logistics/log.log", 'a') as f:
            f.write(str(lr_log_loss) + '\n') 
            
        ##########
        
        # Use CV to train a naive-bayes classifer
        nb = NaiveBayes(maxIter=10, featuresCol='feats_sub',  labelCol = label_name)
        nb_paramGrid = ParamGridBuilder() \
                                        .addGrid(nb.smoothing, [0.8, 1.0])
                                        .build()

        nb_evaluator = MulticlassClassificationEvaluator(labelCol=label_name, metricName='logLoss')
        nb_crossval = CrossValidator(estimator=nb,
                              estimatorParamMaps=nb_paramGrid,
                              evaluator=nb_evaluator,
                              numFolds=3)  

        nb_cvModel = nb_crossval.fit(final_train_df_sub_feats)
        nb_prediction_df = nb_cvModel.transform(final_test_df_sub_feats)

        nb_cvModel.save(f"./naive-bayes/{label_name}.model")

        nb_log_loss = nb_evaluator.evaluate(test_df)

        with open(f"./naive-bayes/log.log", 'a') as f:
            f.write(str(nb_log_loss) + '\n') 

        return (True, log_loss)
    else:
        return (False, None)

In [64]:
train_individual_label = None

Stratify 
ref: https://stackoverflow.com/questions/47637760/stratified-sampling-with-pyspark/47672336

In [21]:
with open(f"./logistics/log.log", 'w') as f:
    pass
with open(f"./naive-bayes/log.log", 'w') as f:
    pass


temp_dict = {}
for name in tqdm(df_label.columns[1:]):
    print(name)
    temp_dict[name] = train_individual_label(df4,  name)
    break

  0%|          | 0/206 [00:00<?, ?it/s]

5-alpha_reductase_inhibitor


KeyboardInterrupt: 

# SCRAP

In [60]:
my_set = {'Geeks', 'for', 'geeks'} 
  
s = list(my_set)

TypeError: 'list' object is not callable

In [62]:
type(list)

list

In [ ]:
cell_feature_names

In [32]:
df.select("cp_type").distinct().show()

+-------+
|cp_type|
+-------+
| trt_cp|
+-------+



In [ ]:
fractions = df.select("x1").distinct().withColumn("fraction", lit(0.8)).rdd.collectAsMap()


In [6]:
from pyspark.sql.functions import lit


In [7]:
lit(0.8)

Column<b'0.8'>

In [8]:
from pyspark.sql.functions import lit
list = [(2147481832,23355149,1),(2147481832,973010692,1),(2147481832,2134870842,1),(2147481832,541023347,1),(2147481832,1682206630,1),(2147481832,1138211459,1),(2147481832,852202566,1),(2147481832,201375938,1),(2147481832,486538879,1),(2147481832,919187908,1),(214748183,919187908,1),(214748183,91187908,1)]
df = spark.createDataFrame(list, ["x1","x2","x3"])

In [9]:
df

DataFrame[x1: bigint, x2: bigint, x3: bigint]

In [10]:
fractions = df.select("x1").distinct().withColumn("fraction", lit(0.8)).rdd.collectAsMap()


In [12]:
fractions

{2147481832: 0.8, 214748183: 0.8}

In [15]:
df.select("x1").distinct().withColumn("fraction", lit(0.8)).show()

+----------+--------+
|        x1|fraction|
+----------+--------+
|2147481832|     0.8|
| 214748183|     0.8|
+----------+--------+



In [ ]:
temp_df = features_and_targets.withColumn('target_vector', (vector_to_string(array([features_and_targets[col] for col in target_names])))).select(['sig_id', 'target_vector'])
string_indexer = StringIndexer(inputCol = 'target_vector', outputCol = 'target')
string_indexer_model = string_indexer.fit(temp_df)
temp_df = string_indexer_model.transform(temp_df).drop('target_vector')

data = features_and_targets.join(temp_df, features_and_targets.sig_id == temp_df.sig_id, how = 'inner').drop(temp_df.sig_id)